In [1]:
import pandas as pd
import numpy as np
from datetime import *
from copy import deepcopy
from iFinDPy import *
import warnings
warnings.filterwarnings('ignore')
# 输入同花顺账户的 账号、密码
THS_iFinDLogin('xxxxxxx','xxxxxx')

c:\Users\huangrz\AppData\Local\Programs\Python\Python311\Lib\site-packages\iFinDPy.pth


0

In [2]:
# cb为所有未到期可转债的日收盘价，st为所有沪深股票的日收盘价
cb = pd.read_csv('C:\\Users\\huangrz\\Desktop\\cb_p\\cb_close.csv',
                 index_col=[0],
                 encoding='GBK',
                 low_memory=False)
st = pd.read_csv('C:\\Users\\huangrz\\Desktop\\cb_p\\st_close.csv',
                 index_col=[0],
                 encoding='GBK',
                 low_memory=False)
cb.drop('日期',axis=1,inplace=True)
cb.index.name = ''
cb.columns = pd.to_datetime(cb.columns)
st.columns = pd.to_datetime(st.columns)
st = st[cb.columns]

In [3]:
# 通过iFinDPy构造可转债代码到正股代码的映射：cb_st_dict
datastruct = THS_BasicData(','.join(cb.index),
                'ths_stock_code_cbond','')
cb_st_dict = {}
struct = datastruct['tables']
for i in range(len(struct)):
    cb_st_dict[struct[i]['thscode']] \
        = struct[i]['table']['ths_stock_code_cbond'][0]
cb_st_list = list(cb_st_dict.items())
cbst = deepcopy(cb)
for i in range(cb.shape[0]):
    cbcode = cb.iloc[i].name
    stcode = cb_st_dict[cbcode]
    if stcode in st.index:
        cbst.iloc[i] = st.loc[stcode]
    else:
        cbst.iloc[i] = np.full(cbst.shape[1],np.nan)

# 将转债正股的收盘价行情数据保存为：cbst.xlsx
cbst.to_excel('cbst.xlsx')

In [4]:
# 根据 observe_window 调整“转股价格”的更新频率，取为 20 基本能覆盖各种
# 触发转股价格调整的事件(转增股/红股分配/现金红利分配/执行向下修正条款)
observe_window = 20
conversion_prices = []
for date in cb.columns[::observe_window]:
    date = date.strftime("%Y%m%d")
    conversion_price_dict = {}
    datastruct = THS_BasicData(','.join(cb.index),
                    'ths_conversion_clause_price_cbond',date)
    struct = datastruct['tables']
    for i in range(len(struct)):
        conversion_price_dict[struct[i]['thscode']] \
            = struct[i]['table']['ths_conversion_clause_price_cbond'][0]
    conversion_prices.append(pd.DataFrame([conversion_price_dict]).T)
conversion_price_df = pd.concat(conversion_prices,axis=1)
conversion_price_df.columns = cb.columns[::observe_window]
conversion_price_df.fillna(np.nan,inplace=True)

# 将观测到的转股价格保存为：conversion_price_df.xlsx
conversion_price_df.to_excel('conversion_price_df.xlsx')

In [5]:
# 赎回(call)、向下修正(reset)、回售(put)的正股触发价比例是在发行时就
# 确定的，在存续期内是一个常数，对应的赎回、下修、回售价格则根据转股价
# 的改变而改变
crdatastruct = THS_BasicData(','.join(cb.index),
                    'ths_resale_trigger_ratio_cbond','')
rrdatastruct = THS_BasicData(','.join(cb.index),
                    'ths_trigger_ratio_cbond','')
prdatastruct = THS_BasicData(','.join(cb.index),
                    'ths_redemp_trigger_ratio_cbond','')
call_ratio_list = list()
reset_ratio_list = list()
put_ratio_list = list()
crstruct = crdatastruct['tables']
rrstruct = rrdatastruct['tables']
prstruct = prdatastruct['tables']
for i in range(len(crstruct)):
    put_ratio_list.append((crstruct[i]['thscode'],
        crstruct[i]['table']['ths_resale_trigger_ratio_cbond'][0]))
    reset_ratio_list.append((rrstruct[i]['thscode'],
        rrstruct[i]['table']['ths_trigger_ratio_cbond'][0]))
    call_ratio_list.append((prstruct[i]['thscode'],
        prstruct[i]['table']['ths_redemp_trigger_ratio_cbond'][0]))
prdf = pd.DataFrame(put_ratio_list,
                    columns=['','put_ratio'])
rrdf = pd.DataFrame(reset_ratio_list,
                    columns=['','reset_ratio'])
crdf = pd.DataFrame(call_ratio_list,
                    columns=['','call_ratio'])
prdf.set_index('',inplace=True)
rrdf.set_index('',inplace=True)
crdf.set_index('',inplace=True)
rdf = pd.concat([prdf,rrdf,crdf],axis=1)

# 将各支转债强赎/下修/回售条款给到的触发比例保存在一个表内：ratios.xlsx
rdf.to_excel('ratios.xlsx')

In [6]:
# 转债上市日和转股起始日一般都不一样，为了对冲策略实际可执行，需要记录每支
# 转债转股起始日的日期，之后以因子(0和1)的形式参与策略构造
datastruct = THS_BasicData(','.join(cb.index),
                'ths_conversion_sd_cbond','')
conversion_sd_list = []
struct = datastruct['tables']
for i in range(len(struct)):
    conversion_sd_list\
        .append((struct[i]['thscode'],
                 struct[i]['table']['ths_conversion_sd_cbond'][0]))
conversion_sd_df \
    = pd.DataFrame(conversion_sd_list, 
                   columns=['bond','conversion date'])
conversion_sd_df['conversion date'] \
    = pd.to_datetime(conversion_sd_df['conversion date'])
conversion_sd_factor \
    = pd.DataFrame(np.zeros(cb.shape),
                   index=cb.index,
                   columns=cb.columns)
conversion_sd_factor \
    = conversion_sd_factor\
        .apply(lambda x:x.mask(\
            x.index>=conversion_sd_df\
                .loc[conversion_sd_df['bond']==x.name,
                     'conversion date'].values[0], 1 ), 
                axis=1)

# 将转股起始日因子保存为：conversion_sd_factor.xlsx
conversion_sd_factor.to_excel('conversion_sd_factor.xlsx')

In [7]:
# # 如果要扩大研究样本，除了未到期合约，还可以将已到期合约引入研究
# start = '2013-01-01'
# end = '2023-08-28'
# tds=THS_DateQuery('SSE','dateType:0,period:D,dateFormat:0',start,end)
# trade_calendars = tds['tables']['time']

# T = len(trade_calendars)
# num = T//120+1
# slices = []
# for i in range(120):
#     s, e = i*num, min((i+1)*num,T)
#     if s>T:break
#     slices.append(slice(s,e))
# time_stamps = [trade_calendars[s][0] for s in slices]

# cb_code_set = set()
# # 记录未到期可转债合约的代码
# cb_code_set = cb_code_set\
#     .union(THS_DP('block',time_stamps[-1]+';031026_640007',
#                   'date:Y,thscode:Y,security_name:Y').data['THSCODE'])
# # 记录已到期可转债合约的代码
# cb_code_set = cb_code_set\
#     .union(THS_DP('block',time_stamps[-1]+';031014002004',
#                   'date:Y,thscode:Y,security_name:Y').data['THSCODE'])
# cb_code = ','.join(cb_code_set)

# # 存续因子:listing_factor
# # 计算纯债价值(因子)时发现存续因子用处不算大
# lds = THS_BasicData(cb_code,'ths_listed_date_cbond','')['tables']
# nlds = THS_BasicData(cb_code,'ths_stop_listing_date_bond','')['tables']
# invalid_cbcode = {}
# listing_interval = [] 
# for i in range(len(lds)):
#     struct = lds[i]
#     nstruct = nlds[i]
#     if struct['thscode']==nstruct['thscode']:
#         sdate = struct['table']['ths_listed_date_cbond'][0]
#         edate = nstruct['table']['ths_stop_listing_date_bond'][0]
#         listing_interval.append((struct['thscode'],sdate,edate))
# listing_interval_df = \
#     pd.DataFrame(listing_interval,
#                  columns=['code','startdate','enddate'])
# listing_interval_df['startdate'] = \
#     pd.to_datetime(listing_interval_df['startdate'])
# listing_interval_df['enddate'] = \
#     pd.to_datetime(listing_interval_df['enddate'])
# listing_interval_df.dropna(inplace=True)
# listing_interval_df.reset_index(drop=True,inplace=True)
# fclose = pd.read_csv('st_close.csv', index_col=[0] ,low_memory=False)
# fclose.columns = pd.to_datetime(fclose.columns)
# listing_factor = pd.DataFrame(np.zeros((listing_interval_df.shape[0],
#                                         fclose.shape[1])),
#                               index=listing_interval_df['code'].values,
#                               columns=fclose.columns)
# for i in range(listing_factor.shape[0]):
#     listing_factor.iloc[i,(listing_factor.columns\
#                            <=listing_interval_df.loc[i,'enddate'])\
#                          &(listing_factor.columns\
#                            >=listing_interval_df.loc[i,'startdate'])]=1
#
# # 将存续因子保存为：listing_factor.csv
# listing_factor.to_csv('listing_factor.csv')

# # 记录每个时间戳下各支债券的到期时间 T
# def rsort_nonzero(x):
#     x[x>0] = -np.sort(-x[x>0])
#     return x
# tmpf = listing_factor.cumsum(axis=1).apply(rsort_nonzero,axis=1)

In [8]:
# # 使用iFinDPy抓取债券的券息，得到的是统一的字符串，部分代码的数据有缺失
# rate = THS_BasicData(cb_code,'ths_nominal_interest_explain_cbond','')
# rts = rate['tables']
# rate_strs = [rts[i]['table']['ths_nominal_interest_explain_cbond'][0]\
#              for i in range(len(rts))]
# rate_dfs=[]
# invalid_indexs=[]
# for i,rate_str in enumerate(rate_strs):
#     rate_ls = []
#     for x in rate_str.split(';'):
#         try:
#             y,z=x.split(',')
#             y1,y2=y.split('-')
#             _,y3=z.split(':')
#             num,_=y3.split('%')
#             rate_ls.append((y1,y2,num))
#         except:
#             invalid_indexs.append(i)
#             continue
#     rate_df = \
#         pd.DataFrame(rate_ls,columns=['start','end','nominal rate(%)'])
#     rate_df['code'] = rts[i]['thscode']
#     rate_df.loc[:,'start'] = pd.to_datetime(rate_df.loc[:,'start'])
#     rate_df.loc[:,'end'] = pd.to_datetime(rate_df.loc[:,'end'])
#     rate_df.loc[:,'nominal rate(%)'] = \
#         rate_df.loc[:,'nominal rate(%)'].astype(np.float64)
#     rate_dfs.append(rate_df)

# # 计算现金流的函数，每一期的券息支付都是在rate_df标注的期初完成，折现率
# # 使用 spot_rate.csv(同花顺Excel插件得到的国债到期收益率：6m+1~10y)
# def calc_cash_flow(cbr,d):
#     if d<cbr['start'].values[0] or d>=cbr['end'].values[-1]:
#         return 0
#     else:
#         cf = sum(cbr['start'].apply(lambda x: (x-d).days/365)\
#                              .apply(lambda x: np.e**(
#                                     -x*spot.loc[np.floor(x),d]*0.01
#                                     )\
#                                     if x>0 else 0)\
#                 *cbr['nominal rate(%)'])
#         return cf

# # 计算券息价值时先遍历每个转债标的，cbr是转债i的 |券息|期初|期末| 的df数据
# # 再遍历每个时间戳运行calc_cash_flow函数
# spot=pd.read_csv('spot_rate.csv',index_col=[0],encoding='GBK').T
# spot.columns=pd.to_datetime(spot.columns)
# spot.index = [i for i in range(0,11)]
# spot=spot[np.sort(spot.columns)]
# spot.columns.name=None
# coupon_factor = \
#     pd.DataFrame(np.zeros((listing_interval_df.shape[0],fclose.shape[1])),
#                  index=listing_interval_df['code'].values,
#                  columns=fclose.columns)
# for i in range(len(rate_dfs)):
#     if len(rate_dfs[i])==0:
#         continue
#     cbr = rate_dfs[i]
#     l = []
#     for d in listing_factor.columns:
#         l.append(calc_cash_flow(cbr,d))
#     arr=np.array(l)
#     code = cbr['code'][0]
#     coupon_factor.loc[code,:]=arr

# # 每个日期下，对转债计算的券息现金流根据该期spot折现，保存为：coupon_factor.csv
# coupon_factor.to_csv('coupon_factor.csv')